In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import contextily as cx
%matplotlib inline

import math

import os
import urllib.request
from PIL import Image

In [36]:
'''
    Will return the x and y tile for a given latitude and longitude pair
    along with the zoom.
    :param lat_deg: float, latitude in degrees
    :param lon_deg: float, longitude in degrees
    :param zoom: integer, zoom for image
    :return: zoom of tile, xtile, ytile
'''


def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
    return (zoom, xtile, ytile)

In [51]:
# Setting up the necessary directories in order to save off images correctly

folder_name = "DM_Dataset"

os.makedirs("Collected_Images", exist_ok=True)
os.makedirs("Collected_Images/%s" % folder_name, exist_ok=True)
os.makedirs("Collected_Images/%s/Formation_pics" % folder_name, exist_ok=True)
os.makedirs("Collected_Images/%s/SAT_pics" % folder_name, exist_ok=True)
os.makedirs("Collected_Images/%s/GAN_pics" % folder_name, exist_ok=True)

# Setting up save paths that were established in the directories
save_paths = [
                'Collected_Images/%s/SAT_pics/' % folder_name,
                'Collected_Images/%s/Formation_pics/' % folder_name,
                'Collected_Images/%s/GAN_pics/' % folder_name
             ]

# Constructing the mapbox api request to access the satelite tiles
first = "https://api.mapbox.com/styles/v1/mapbox/satellite-v9/tiles/"
second = "%d/%d/%d?access_token="
access_token = "pk.eyJ1IjoiY2xhd3NvbjEwMSIsImEiOiJja2Iwdzg3cHMwMHZvMnZvODBncDRjbjBiIn0.PeYOCiVXGZ_7oJLvKFL28Q"

url = first+second+access_token

# Organizing the url requests that are going to be called for each
# area. The first url is the satelite image request, the second is the
# macrostrat request for the formation images.

urls = [
            url,
            "https://tiles.macrostrat.org/carto/%d/%d/%d.png"
       ]

In [52]:
def nearest_to(lat, lng, z, t_x, t_y):
    z, cur_x, cur_y = deg2num(lat, lng, z)
    temp_lat = lat
    temp_lng = lng
    in_lat = False
    in_lng = False
    while(not (in_lat and in_lng)):
        if(cur_x < t_x):
            temp_lng += 0.0001
        elif(cur_x > t_x):
            temp_lng -= 0.0001
            
        if(cur_y < t_y):
            temp_lat += 0.0001
        elif(cur_y > t_y):
            temp_lat -= 0.0001
            
        z, cur_x, cur_y = deg2num(lat, lng, z)
        in_lng = cur_x == t_x
        in_lat = cur_y == t_y
    
    x_trip = False
    y_trip = False
    
    while(not(x_trip and y_trip)):
        
        if(not(x_trip)):
            temp_lng -= 0.0001
        if(not(y_trip)):    
            temp_lat += 0.0001
        
        z, cur_x, cur_y = deg2num(temp_lat, temp_lng, z)
        x_trip = not(cur_x == t_x)
        y_trip = not(cur_y == t_y)
        
    temp_lat -= .0001
    temp_lng += .0001
    
    return (z, temp_lat, temp_lng)
    
    

In [53]:
names_data = [
    #"Nebraska",
    #"Illinois",
    #"Arizona_Phoenix",
    "Chihuahua_N",
    "Alabama",
    "Utah",
    "W_Virginia",
    "New_Mexico_Franklin",
    "Colorodo",
    "W_Texas",
    "Mid_New_Mexico",
    "W_New_Mexico",
    "S_New_Mexico",
    "S_Arizona",
    "S_California",
    "Mid_Nevada",
    "N_Colorodo",
    "S_W_Virginia",
    "Tennessee"
]


lat_data = [
    #41.2448,
    #42.8742,
    #33.4630,
    29.7751,
    31.5821,
    42.5344,
    37.0020,
    33.0691,
    37.6465,
    31.2306,
    35.0975,
    33.3027,
    32.1383,
    33.2574,
    34.4973,
    39.5407,
    40.4605,
    38.2043,
    36.1922,
]

lng_data = [
#      -96.3281,
#      -88.2415,
#     -112.1428,
    -105.2133,
    -105.6609,
    -111.9644,
     -81.5405,
    -106.5550,
    -107.0412,
    -103.2689,
    -104.6570,
    -105.9667,
    -108.2893,
    -113.9307,
    -115.0333,
    -116.5303,
    -107.1524,
    -79.4638,
    -83.8231,
]

z = 10

# for i in range(0, len(names_data)):
#     print("Name: %s\nLat/Lng: %f %f\n" % (names_data[i], lat_data[i], lng_data[i]))

In [54]:
out_lat_data = []
out_lng_data = []

In [55]:
for i in range(0, len(names_data)):
    name = names_data[i]
    lat = lat_data[i]
    lng = lng_data[i]

    z, x, y = deg2num(lat, lng, z)

    z, lat, lng = nearest_to(lat, lng, z, x, y)
    print("Name: %s\nlat: %3.4f\nlng: %3.4f" % (name, lat, lng))

    out_lat_data.append(round(lat,4))
    out_lng_data.append(round(lng,4))

    names = [
                name + "_sat",
                name + "_form"
            ]

    a = 0

    for k in range(0, len(urls)):

        imgs = []

        for i in range (-1,2,1):
            for j in range(-1,2,1):
                imgs.append(Image.open(urllib.request.urlopen(urls[k] % (z,x+i,y+j))))


        w1, h1 = imgs[0].size

        jnt = Image.new("RGB", (w1+w1+w1, h1+h1+h1))

        jnt.paste(imgs[0], (0,0))
        jnt.paste(imgs[1], (0,h1))
        jnt.paste(imgs[2], (0,h1+h1))

        jnt.paste(imgs[3], (w1,0))
        jnt.paste(imgs[4], (w1,h1))
        jnt.paste(imgs[5], (w1,h1+h1))

        jnt.paste(imgs[6], (w1+w1,0))
        jnt.paste(imgs[7], (w1+w1,h1))
        jnt.paste(imgs[8], (w1+w1,h1+h1))

        jnt.save(save_paths[a] + names[k] + ".png")
        a += 1


Name: Chihuahua_N
lat: 29.8406
lng: -105.4687
Name: Alabama
lat: 31.6533
lng: -105.8203
Name: Utah
lat: 42.5530
lng: -112.1484
Name: W_Virginia
lat: 37.1603
lng: -81.5624
Name: New_Mexico_Franklin
lat: 33.1375
lng: -106.8749
Name: Colorodo
lat: 37.7185
lng: -107.2265
Name: W_Texas
lat: 31.3536
lng: -103.3593
Name: Mid_New_Mexico
lat: 35.1738
lng: -104.7656
Name: W_New_Mexico
lat: 33.4314
lng: -106.1718
Name: S_New_Mexico
lat: 32.2499
lng: -108.6328
Name: S_Arizona
lat: 33.4314
lng: -114.2578
Name: S_California
lat: 34.5970
lng: -115.3124
Name: Mid_Nevada
lat: 39.6395
lng: -116.7187
Name: N_Colorodo
lat: 40.7139
lng: -107.2265
Name: S_W_Virginia
lat: 38.2726
lng: -79.8046
Name: Tennessee
lat: 36.3151
lng: -84.0234


In [56]:
lat = [
    40.4605
]

lng = [   
    -107.1524
]

for i in range(0, len(lat)):
    temp = deg2num(lat[i], lng[i], 10)
    print("/carto/10/%s/%s.png       /#10/%s/%s" % (temp[1], temp[2], lat[i], lng[i]))

/carto/10/207/385.png       /#10/40.4605/-107.1524


In [57]:
for name in names_data:
    sat_img = Image.open(save_paths[0] + name + "_sat.png")
    form_img = Image.open(save_paths[1] + name + "_form.png")
    
    wS, hS = sat_img.size
    wF, hF = form_img.size
    
    joint = Image.new("RGB", (wS + wF, hS))
    joint.paste(sat_img, (0,0))
    joint.paste(form_img, (wS,0))
    
    joint.save(save_paths[2] + name + "_gan.png")

In [58]:
crop = 328

for i in range(0, len(names_data)):
    tmp_img_sat = Image.open(save_paths[2] + names_data[i] + "_gan.png")
    tmp_img_frm = Image.open(save_paths[2] + names_data[i] + "_gan.png")

    w, h = tmp_img_sat.size
    left = 0 + crop
    top = 0 + crop
    right = w/2 - crop
    bottom = h - crop

    tmp_img_sat = tmp_img_sat.crop((left, top, right, bottom))

    w, h = tmp_img_frm.size
    left = w/2 + crop
    top = 0 + crop
    right = w - crop
    bottom = h - crop

    tmp_img_frm = tmp_img_frm.crop((left, top, right, bottom))

    save_img = Image.new("RGB", (tmp_img_frm.width + tmp_img_frm.width,tmp_img_frm.height))
    save_img.paste(tmp_img_sat, (0,0))
    save_img.paste(tmp_img_frm, (tmp_img_sat.width, 0))
    save_img.save(save_paths[2] + names_data[i] + "-gan.png")

for i in range(0, len(names_data)):
    os.remove(save_paths[2] + names_data[i] + "_gan.png")